# Langchain RAG 시스템

이 노트북은 data 폴더의 PDF 파일들을 활용하여 RAG (Retrieval-Augmented Generation) 시스템을 구현합니다.

## 주요 기능
1. PDF 문서 로딩 및 전처리
2. 텍스트 청킹 (chunking)
3. 임베딩 생성 및 벡터 스토어 구축
4. 검색 기반 질문-답변 시스템

## 사용할 데이터(데이터 출처: 위키피디아)
- 귀신고래.pdf
- 범고래.pdf  
- 흰꼬리수리.pdf


## 환경 설정

In [ ]:
!pip install langchain-openai langchain-community chromadb pymupdf -q

In [ ]:
import os
import getpass
from pathlib import Path

from langchain_community.document_loaders import PyMuPDFLoader

In [ ]:
try:
    from google.colab import userdata   
    api_key = userdata.get('OPENAI_API_KEY')
    print("Colab Secrets에서 API 키를 성공적으로 불러왔습니다.")
except:
    import getpass
    api_key = getpass.getpass("OpenAI API 키를 입력하세요: ")
    print("API 키가 입력되었습니다.")

os.environ["OPENAI_API_KEY"] = api_key

## 인덱싱(Indexing)

### 문서 추출

In [ ]:
# 데이터 폴더에서 PDF 파일 목록 가져오기
data_path = Path("data")
pdf_files = list(data_path.glob("*.pdf"))

# 모든 PDF 문서를 담을 리스트
all_documents = []

# 각 PDF 파일을 순회하며 로드
for pdf_file in pdf_files:
    print(f"불러오는 중 : {pdf_file}")
    loader = PyMuPDFLoader(str(pdf_file))
    documents = loader.load()
    all_documents.extend(documents)

print("파일 유형", type(all_documents[0]))
print("파일 수", len(all_documents))
print(f"페이지 별 글자 수: {[len(x.page_content) for x in all_documents]}")

### 청크 분할(Chunking)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
chunks = splitter.split_documents(all_documents)

print("철자 단위 청킹 후 문서 수", len(chunks))
print("청크 별 글자 수", [len(x.page_content) for x in chunks])
print("-"*50)
print(chunks[0].page_content[600:])
print("-"*50)
print(chunks[1].page_content[:200])

### 임베딩

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Chroma 벡터 스토어 생성 및 저장
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

print(f"벡터 스토어에 {vectorstore._collection.count()}개의 벡터 저장 완료")

In [ ]:
first_id = vectorstore.get()['ids'][0]  # 첫 번째 청크의 id 자동 추출
info = vectorstore.get(ids=[first_id], include=["embeddings", "documents", "metadatas"])

print(f"첫 번째 청크의 인덱스: {info['ids']}")
print(f"청크 임베딩 차원: {info['embeddings'][0].shape}\n값: {info['embeddings'][0][:10]}...")
print(f"청크 문서: {info['documents'][0]}")
print(f"청크 메타데이터: {info['metadatas'][0]}")

In [ ]:
vectorstore.search(query="범고래의 먹이",
                   search_type="similarity_score_threshold")

### 검색기 생성

In [ ]:
# 검색기(Retriever) 생성
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

retriever.invoke("범고래의 먹이")

## RAG 체인 구성

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# LLM 초기화
llm = ChatOpenAI(
    model="gpt-5-nano",
    temperature=0,
)

# 프롬프트
messages = [
    ("system", "다음 문맥을 사용하여 질문에 답하세요. 문맥에서 답을 찾을 수 없다면, 모른다고 말하세요."),
    ("human", "문맥: {context}\n질문: {question}."),
]

prompt = ChatPromptTemplate.from_messages(messages)


# 검색된 문서 리스트를 하나의 문자열로 합치는 함수
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 체인 생성
rag_chain = (
    # context와 question을 딕셔너리 형태로 구성
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("범고래는 어떤 먹이를 먹나요?")

In [ ]:
# 또는 아래와 같은 방식으로도 체인 구성 가능
from langchain_classic.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate

# 프롬프트
template = """다음 문맥을 사용하여 질문에 답하세요. 문맥에서 답을 찾을 수 없다면, 모른다고 말하세요.

문맥:
{context}

질문: {question}

답변: 위의 문맥을 바탕으로 질문에 대해 자세하고 정확한 답변을 한국어로 제공해주세요."""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

# RetrievalQA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

In [ ]:
def answer_with_wiki(question):
    print(f"질문: {question}")

    response = qa_chain.invoke({"query": question})
    answer = response["result"]
    print(f"답변: {answer}")
    
    source_documents = response["source_documents"]
    print(f"참고 문서: {source_documents}")

answer_with_wiki("범고래는 어떤 먹이를 먹나요?")